In [19]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [35]:
# 데이터 불러오기
df = pd.read_csv('./data/worldcup_player.csv')  # 기존 데이터셋
df['fifa_score'] = None  # 점수 컬럼 추가

In [36]:
df

,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,fifa_score
0,2002,세네갈,GK,1,토니 실바,Tony Sylva,1975.05.17,AS 모나코,None
1,2002,세네갈,GK,16,오마르 디알로,Omar Diallo,1972.09.28,올랭피크 쿠리브가,None
2,2002,세네갈,GK,22,칼리두 시소코,Kalidou Cissokho,1978.08.28,ASC 잔다르크,None
3,2002,세네갈,DF,2,오마르 다프,Omar Daf,1977.02.12,FC 소쇼,None
4,2002,세네갈,DF,4,파페 말리크 디오프,Pape Malick Diop,1974.12.29,FC 로리앙,None
...,...,...,...,...,...,...,...,...,...
4506,2014,브라질,FW,7,헐크,Hulk,1986.07.25,FC 제니트 상트페테르부르크,None
4507,2014,브라질,FW,9,프레드,Fred,1983.10.03,플루미넨시 FC,None
4508,2014,브라질,FW,10,네이마르 주니오르,Neymar,1992.02.05,FC 바르셀로나,None
4509,2014,브라질,FW,20,베르나르드,Bernard,1992.09.08,FC 샤흐타르 도네츠크,None


In [ ]:


# Selenium 설정
options = Options()
options.add_argument("--headless")  # 브라우저 안띄우기
driver = webdriver.Chrome(options=options)

# FIFA 버전 매핑
fifa_version_map = {
    2022: '21',
    2018: '17',
    2014: '13',
    2010: '09',
    2006: '07'
}

features = ['속도 / 다이빙',
'슛 / 핸들링',
'패스 / 킥',
'드리블 / 반응속도',
'수비 / 속도',
'피지컬 / 위치선정']

driver = webdriver.Chrome()
driver.get(f'https://sofifa.com/?hl=ko-KR')
time.sleep(2)
element=driver.find_element(By.CSS_SELECTOR,'div.choices.choices-multiple')
element.click()
search_box=driver.find_element(By.CSS_SELECTOR,'input.choices-input.choices-input-cloned')
for i in features:
    search_box.send_keys(i)
    search_box.send_keys(Keys.ENTER)
element = driver.find_element(By.TAG_NAME, "body")  # body에 보내도 됨
element.send_keys(Keys.ESCAPE)
button = driver.find_element(By.XPATH,'//*[@id="body"]/main[1]/aside/form[1]/div[2]/div/button')
button.click()
prev=0
for idx, row in df.iterrows():
    year = row['년도']
    if year == 2002:
        df.at[idx, 'fifa_score'] = None
        continue
    player_name = row['로마자 성명']
    # FIFA 버전 선택 (드롭다운)
    if year in fifa_version_map and prev!=year:
        try:
            select = Select(driver.find_element(By.ID, 'select-version'))
            select.select_by_visible_text("FIFA "+fifa_version_map[year])
            time.sleep(1)  # 선택 후 로딩 대기
            prev = year
        except NoSuchElementException:
            pass
    name_box=driver.find_element(By.CSS_SELECTOR,'input.choices-input.choices-input-cloned')    
    name_box.send_keys(row['로마자 성명'])
    name_box.send_keys(Keys.ENTER)
    time.sleep(0.2)  # 페이지 로딩 대기
    # 첫 번째 검색 결과 가져오기
    # try:
    #     rating = driver.find_element(By.XPATH, '//*[@id="body"]/main[1]/article/table').text
    #     df.at[idx, 'fifa_score'] = rating
    # except NoSuchElementException:
    #     df.at[idx, 'fifa_score'] = None

driver.quit()

df.to_csv('worldcup_players_with_fifa.csv', index=False)


In [55]:
features = ['속도 / 다이빙',
'슛 / 핸들링',
'패스 / 킥',
'드리블 / 반응속도',
'수비 / 속도',
'피지컬 / 위치선정']

driver = webdriver.Chrome()
driver.get(f'https://sofifa.com/?hl=ko-KR')
time.sleep(2)
element=driver.find_element(By.CSS_SELECTOR,'div.choices.choices-multiple')
element.click()
search_box=driver.find_element(By.CSS_SELECTOR,'input.choices-input.choices-input-cloned')
for i in features:
    search_box.send_keys(i)
    search_box.send_keys(Keys.ENTER)
element = driver.find_element(By.TAG_NAME, "body")  # body에 보내도 됨
element.send_keys(Keys.ESCAPE)
button = driver.find_element(By.XPATH,'//*[@id="body"]/main[1]/aside/form[1]/div[2]/div/button')
button.click()

In [56]:
select = Select(driver.find_element(By.ID, 'select-version'))

select.select_by_visible_text("FIFA 13")

In [59]:
name_box=driver.find_element(By.XPATH,'//*[@id="body"]/main[1]/aside/form[2]/div[1]/input') 
name_box.send_keys('Fred')
nation_box=driver.find_element(By.XPATH,'//*[@id="body"]/main[1]/aside/form[2]/div[4]/div[1]/input')
nation_box.send_keys('Brazil')
nation_box.send_keys(Keys.ENTER)
b = driver.find_element(By.XPATH,'//*[@id="body"]/main[1]/aside/form[2]/div[19]/button')
b.click()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (269, 1211)
  (Session info: chrome=140.0.7339.128); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6808e30f5+79493]
	GetHandleVerifier [0x0x7ff6808e3150+79584]
	(No symbol) [0x0x7ff6806601ba]
	(No symbol) [0x0x7ff6806c01f9]
	(No symbol) [0x0x7ff6806bdb7e]
	(No symbol) [0x0x7ff6806baac1]
	(No symbol) [0x0x7ff6806b99b0]
	(No symbol) [0x0x7ff6806aaf58]
	(No symbol) [0x0x7ff6806e070a]
	(No symbol) [0x0x7ff6806aa7e6]
	(No symbol) [0x0x7ff6806e0920]
	(No symbol) [0x0x7ff680708b8b]
	(No symbol) [0x0x7ff6806e04e3]
	(No symbol) [0x0x7ff6806a8e92]
	(No symbol) [0x0x7ff6806a9c63]
	GetHandleVerifier [0x0x7ff680ba0dbd+2954061]
	GetHandleVerifier [0x0x7ff680b9b02a+2930106]
	GetHandleVerifier [0x0x7ff680bbb357+3061991]
	GetHandleVerifier [0x0x7ff6808fd60e+

In [54]:
driver.quit()

In [2]:
import pandas as pd

In [31]:
df1 = pd.read_csv('./data/21.csv',sep=';')
df2 = pd.read_csv('./data/24.csv')

In [35]:
df1[df1['Fullname']=='Neymar Jr']['shot_power']


14832    80
Name: shot_power, dtype: int64

In [34]:
df2[df2['Name']=='Neymar Jr']['Shooting']

12    83
Name: Shooting, dtype: int64

In [7]:
df1.columns

Index(['id', 'Fullname', 'current_rating', 'potential_rating', 'height',
       'weight', 'preferred_foot', 'birth_date', 'preferred_positions',
       'work_rate', 'weak_foot', 'skill_moves', 'value', 'wage',
       'ball_control', 'dribbling', 'marking', 'tackling', 'aggression',
       'composure', 'reaction', 'crossing', 'short_pass', 'long_pass',
       'acceleration', 'stamina', 'strength', 'spring_speed', 'heading',
       'shot_power', 'finishing', 'long_shots', 'FK_Acc.', 'reflexes',
       'handling', 'gk_positioning', 'gk_diving'],
      dtype='object')

In [30]:
df2['Ball'].describe()
df1['gk_positioning'].describe()

count    10912.000000
mean        15.324963
std         17.059643
min          1.000000
25%          7.000000
50%         10.000000
75%         13.000000
max         96.000000
Name: gk_positioning, dtype: float64

In [8]:
df2.columns

Index(['Unnamed: 0', 'Name', 'Nation', 'Club', 'Position', 'Age', 'Overall',
       'Pace', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physicality',
       'Acceleration', 'Sprint', 'Positioning', 'Finishing', 'Shot', 'Long',
       'Volleys', 'Penalties', 'Vision', 'Crossing', 'Free', 'Curve',
       'Agility', 'Balance', 'Reactions', 'Ball', 'Composure', 'Interceptions',
       'Heading', 'Def', 'Standing', 'Sliding', 'Jumping', 'Stamina',
       'Strength', 'Aggression', 'Att work rate', 'Def work rate',
       'Preferred foot', 'Weak foot', 'Skill moves', 'URL', 'Gender', 'GK'],
      dtype='object')

current_rating == Overall
dribbling == 'Dribbling'
composure == 'Composure'
'short_pass', 'long_pass' 평균 ='Passing'
'stamina' == 'Stamina'
'strength' == 'Strength'
'marking', 'tackling' 평균 = 'Defending'
'crossing' = 'Crossing'
'acceleration' ==  'Acceleration'
'reaction' == 'Reactions'
'ball_control' = 'Ball'
'gk_positioning', 'gk_diving' , 'GK'
'spring_speed' = 'Sprint'
'shot_power' = 'Shooting'